In [ ]:
from google.cloud import bigquery
import pandas as pd

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
client = bigquery.Client()

In [ ]:
from datetime import timedelta  

In [ ]:
asset_num = 50;

In [ ]:
def get_first_timestamp():
    query_job = client.query("""SELECT Min(close_timestamp_ms) AS first_timestamp
FROM Binance_1m_BTC_klines.Binance_1m_BTC_1500005000_1548440000;""")
    first_timestamp = query_job.result()
    for row in first_timestamp:
        return row.first_timestamp

In [ ]:
def get_last_timestamp():
    query_job = client.query("""SELECT Max(close_timestamp_ms) AS last_timestamp
FROM Binance_1m_BTC_klines.Binance_1m_BTC_1500005000_1548440000;""")
    last_timestamp = query_job.result()
    for row in last_timestamp:
        return row.last_timestamp

In [ ]:
# todo minus day on last timestamp plus day on first timestamp

In [ ]:
def get_top_assets_for_period(asset_num, start_time, end_time):
    top_symbols = []
    query_job = client.query("""
    SELECT base_asset, SUM(CAST(quote_asset_volume AS FLOAT64)) AS total_volume 
    FROM Binance_1m_BTC_klines.Binance_1m_BTC_1500005000_1548440000
    WHERE close_timestamp_ms>=1548346400000 and close_timestamp_ms<=1548440000000
    GROUP BY base_asset ORDER BY total_volume DESC LIMIT 50""")
    top_assets = query_job.result()
    for row in top_assets:
#         print("{} : {}".format(row.base_asset, row.total_volume))
        top_symbols.append(row.base_asset)
    return top_symbols;

In [ ]:
top_symbols = get_top_assets_for_period()

In [ ]:
first_timestamp = get_first_timestamp()

In [ ]:
last_timestamp = get_last_timestamp()

In [ ]:
timestamp_range = last_timestamp - first_timestamp

In [ ]:
first_timestamp = first_timestamp + (60000*1440)

In [ ]:
end_timestamp = first_timestamp + (60000*1440)

In [ ]:
last_timestamp = last_timestamp - (60000*1440)

In [ ]:
start_timestamp = last_timestamp - (60000*1440)*50

In [ ]:
1548353520000 - start_timestamp

In [ ]:
# todo round timestamp_ms

In [ ]:
def get_features(start_time, end_time, selection_start_time, selection_end_time):
    query = """
    SELECT
    base_asset,
    timestamp_ms,
    CAST(close AS FLOAT64) AS c,
    CAST(open AS FLOAT64) AS o,
    CAST(low AS FLOAT64) AS l,
    CAST(high AS FLOAT64) AS h,
    CAST(quote_asset_volume AS FLOAT64) AS v,
    CAST(trades AS FLOAT64) AS t
    FROM Binance_1m_BTC_klines.Binance_1m_BTC_1500005000_1548440000
    WHERE base_asset IN (
        SELECT base_asset
        FROM Binance_1m_BTC_klines.Binance_1m_BTC_1500005000_1548440000
        WHERE close_timestamp_ms>={sts} AND close_timestamp_ms<={ets}
        GROUP BY base_asset ORDER BY SUM(CAST(quote_asset_volume AS FLOAT64)) DESC LIMIT 50
    )
    AND timestamp_ms>={st}
    AND timestamp_ms<={et}+1
    ORDER BY base_asset,timestamp_ms;
    """.format(st=start_time, et=end_time, sts=selection_start_time, ets=selection_end_time)
    query_job = client.query(query)
    features = query_job.to_dataframe()
    return features

In [ ]:
features = get_features(start_timestamp, last_timestamp, start_timestamp, last_timestamp)

In [ ]:
x = features.copy()
x.set_index(['base_asset', 'timestamp_ms'], inplace=True)

In [ ]:
t_index = list(range(int(start_timestamp/1000), int(last_timestamp/1000), 60))
time_index = pd.to_datetime(t_index,unit='s')
time_index = time_index.round('min');

In [584]:
y = features.copy()

In [585]:
y['timestamp_ms'] = pd.to_datetime(y['timestamp_ms'], unit='ms')
y.set_index(['base_asset', 'timestamp_ms'], inplace=True)
y=y.groupby(['base_asset','timestamp_ms']).first()
ind = pd.MultiIndex.from_product([y.index.levels[0], time_index], names=y.index.names)
y = y.reindex(ind)
y = y.fillna(axis=1, method="bfill").fillna(axis=1, method="ffill")

In [588]:
m = y[['c','h','l']]

In [589]:
res = m.to_xarray()

In [590]:
res

<xarray.Dataset>
Dimensions:       (base_asset: 50, timestamp_ms: 2880)
Coordinates:
  * base_asset    (base_asset) object 'ARN' 'BCHABC' 'BCHSV' ... 'ZIL' 'ZRX'
  * timestamp_ms  (timestamp_ms) datetime64[ns] 2019-01-22T18:12:00 ... 2019-01-24T18:11:00
Data variables:
    c             (base_asset, timestamp_ms) float64 7.976e-05 ... 8.634e-05
    h             (base_asset, timestamp_ms) float64 7.983e-05 ... 8.64e-05
    l             (base_asset, timestamp_ms) float64 7.969e-05 ... 8.634e-05

In [567]:
arr = res.to_array()

In [ ]:
res[:, :, 50:50+90+1].shape

In [ ]:
# portfolio vector memory, [time, assets]
PVM = pd.DataFrame(
    index=y.index.levels[1],
    columns=y.index.levels[0]
)

In [ ]:
PVM = PVM.fillna(1.0 / len(y.index.levels[0]))